In [3]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline

from tyssue.core.sheet import Sheet

from tyssue import config

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.solvers.sheet_vertex_solver import Solver as solver
from tyssue.dynamics.sheet_vertex_model import SheetModel as model


from tyssue.draw.vispy_draw import vp_view

from tyssue.draw.plt_draw import sheet_view
from tyssue.stores import load_datasets

from tyssue.topology.sheet_topology import remove_face
from tyssue.core.objects import get_opposite

In [65]:
datasets = load_datasets('small_hexagonal.hf5',
                         data_names=['face', 'vert', 'edge'])
datasets['edge'] = datasets['edge'][['srce', 'trgt', 'face']]
datasets['face'] = datasets['face'][['x', 'y', 'z', 'is_alive']]
datasets['vert'] = datasets['vert'][['x', 'y', 'z']]

specs = config.geometry.sheet_spec()
sheet = Sheet('emin', datasets, specs)
geom.update_all(sheet)

boundary_cells = sheet.face_df[sheet.face_df.is_alive==0].index
for cell in boundary_cells:
    remove_face(sheet, cell)

sheet.reset_index()
sheet.edge_df['opposite'] = get_opposite(sheet.edge_df)

face_z = sheet.upcast_face(sheet.face_df['z'])

left = face_z[face_z<0].index
left_border = sheet.edge_df.loc[left][sheet.edge_df.loc[left]['opposite']==-1]

right = face_z[face_z>0].index
right_border = sheet.edge_df.loc[right][sheet.edge_df.loc[right]['opposite']==-1]



left_cell_edges = left_border.copy()
right_cell_edges = right_border.copy()

left_face = sheet.face_df.index.max()+1
right_face = sheet.face_df.index.max()+2

left_cell_edges[['srce', 'trgt']] = left_cell_edges[['trgt', 'srce']]
left_cell_edges['face'] = left_face

right_cell_edges[['srce', 'trgt']] = right_cell_edges[['trgt', 'srce']]
right_cell_edges['face'] = right_face


left_vert = sheet.vert_df.loc[left_cell_edges['srce']]
right_vert = sheet.vert_df.loc[right_cell_edges['srce']]

new_faces = pd.DataFrame(data=0,
                         index=[left_face, right_face],
                         columns=sheet.face_df.columns)

sheet.edge_df = sheet.edge_df.append(left_cell_edges)
sheet.edge_df = sheet.edge_df.append(right_cell_edges)
sheet.face_df = sheet.face_df.append(new_faces)
sheet.face_df['is_alive'] = 1


sheet.reset_topo()
sheet.reset_index()

geom.update_all(sheet)

In [51]:
vp_view(sheet)

In [59]:
sheet.face_df = sheet.face_df.loc[sheet.face_df.index.delete(55)]

sheet.face_df.tail()

,x,y,z,is_alive,area,num_sides,perimeter,vol
face,,,,,,,,
52,4.019221e+00,-4.019221e+00,-7.401487e-17,1,18.865118,6.0,16.239088,37.502048
53,5.251365e+00,-2.175186e+00,4.072821e+00,1,18.865118,6.0,16.239088,37.502048
54,3.617298e+00,-4.216089e+00,7.874121e+00,1,18.669681,5.0,17.078073,36.083223
56,-1.554312e-15,6.106227e-16,-1.018205e+01,1,97.553434,8.0,35.959068,182.702784
57,-1.637579e-15,4.440892e-16,1.018205e+01,1,97.553434,8.0,35.959068,182.702784


In [60]:
from tyssue.topology.sheet_topology import cell_division

In [62]:
sheet.edge_df

,srce,trgt,face,nx,length,ny,dx,nz,dz,dy,sub_area,sub_vol,opposite
0,78.0,64.0,1.0,1.017455e+01,4.494884,2.023845e+00,-0.876908,-4.018821e-01,0.000000e+00,4.408516e+00,5.190834,9.721644,0.0
1,64.0,2.0,1.0,7.395163e+00,3.589430,7.284291e-01,0.649007,-1.205646e+00,3.394018e+00,-9.713080e-01,3.764061,7.049515,28.0
2,0.0,78.0,1.0,7.969985e+00,3.589430,2.239684e-01,0.227901,6.107736e-01,-3.394018e+00,1.145736e+00,3.998246,7.948129,196.0
3,1.0,0.0,1.0,6.174476e+00,2.702165,-5.044608e-01,-0.455802,-1.221547e+00,-1.357607e+00,-2.291472e+00,3.157167,6.276145,201.0
4,2.0,1.0,1.0,5.024830e+00,2.702165,5.044608e-01,0.455802,-8.355660e-01,1.357607e+00,-2.291472e+00,2.559373,5.087789,5.0
5,1.0,2.0,2.0,6.063967e+00,2.702165,1.618731e+00,-0.455802,6.963050e-01,-1.357607e+00,2.291472e+00,3.157405,6.276619,4.0
6,3.0,1.0,2.0,5.906097e+00,2.715214,1.999863e+00,0.000000,-1.958579e-15,-2.715214e+00,-2.659164e-15,3.117749,6.197786,203.0
7,2.0,6.0,2.0,5.432488e+00,2.702165,3.143256e+00,-1.298014,6.963050e-01,1.357607e+00,1.942616e+00,3.157405,6.276619,30.0
8,4.0,3.0,2.0,6.063967e+00,2.702165,1.618731e+00,0.455802,-6.963050e-01,-1.357607e+00,-2.291472e+00,3.157405,6.276619,11.0
9,5.0,4.0,2.0,5.432488e+00,2.702165,3.143256e+00,1.298014,-6.963050e-01,1.357607e+00,-1.942616e+00,3.157405,6.276619,39.0


In [61]:
cell_division(sheet, left_face, geom)

KeyError: 'the label [edge] is not in the [columns]'

In [39]:
geom.update_all(sheet)